# Analyzing Instagram trend data

## Our analysis subjects:

1. The most profiles per city?
2. The most posts per city?
3. The most posts per profile?
4. The most likes per profile?


## Publicacao de alteracao

1. git add .\InstagramIndexes.ipynb
2. git commit -m "Mensagem do commit"
3. git status 
4. git push -u origin master

## Verificando versão python

In [ ]:
#Versao da Linguagem Python
from platform import python_version
print('Versao da Linguagem Python Usada neste Jupyter Notebook', python_version())

## Particionando arquivo posts 


In [ ]:
import os
import shutil
import csv

# caminho do arquivo grande
file_path = r'E:\Kaggle\input\Instagram\instagram_posts.csv'

# tamanho de cada arquivo menor em bytes
chunk_size = 10000000

# abrir arquivo grande em modo texto
with open(file_path, 'r', newline='', encoding='utf-8') as f:
    # criar leitor CSV
    reader = csv.reader(f)
    # ler cabeçalho
    header = next(reader)
    # inicializar contador de chunk
    chunk_num = 0
    while True:
        # ler chunk do arquivo
        chunk = ''
        current_size = 0
        for row in reader:
            row_str = ','.join(row) + '\n'
            row_size = len(row_str.encode('utf-8'))
            if current_size + row_size > chunk_size:
                break
            chunk += row_str
            current_size += row_size
        else:
            # fim do arquivo, sair do loop
            if not chunk:
                break
        # nome do arquivo de chunk
        chunk_name = f'{os.path.splitext(file_path)[0]}_{chunk_num}.csv'
        # criar arquivo de chunk e escrever cabeçalho e chunk nele
        with open(chunk_name, 'w', newline='', encoding='utf-8') as chunk_file:
            writer = csv.writer(chunk_file)
            writer.writerow(header)
            chunk_file.write(chunk)
        # incrementar contador de chunk
        chunk_num += 1

## Carregando dataframes e definindo schemas

In [ ]:
#Importando o módulo csv
import pandas as pd
import numpy  as np
from decimal import Decimal

def location_schema():
    schema = pd.DataFrame({
        "sid": pd.Series(dtype="Int64"),
        "id": pd.Series(dtype="Int64"),
        "name": pd.Series(dtype="string"),
        "street": pd.Series(dtype="string"),
        "zip": pd.Series(dtype="string"),
        "city": pd.Series(dtype="string"),
        "region": pd.Series(dtype="string"),
        "cd": pd.Series(dtype="string"),
        "phone": pd.Series(dtype="string"),
        "aj_exact_city_match": pd.Series(dtype="string"),
        "aj_exact_country_match": pd.Series(dtype="string"),
        "blurb": pd.Series(dtype="string"),
        "dir_city_id": pd.Series(dtype="string"),
        "dir_city_name": pd.Series(dtype="string"),
        "dir_city_slug": pd.Series(dtype="string"),
        "dir_country_id": pd.Series(dtype="string"),
        "dir_country_name": pd.Series(dtype="string"),
        "lat": pd.Series(dtype="string"),
        "lng": pd.Series(dtype="string"),
        "primary_alias_on_fb": pd.Series(dtype="string"),
        "slug": pd.Series(dtype="string"),
        "website": pd.Series(dtype="string"),
        "cts": pd.Series(dtype="string")
    })
    return schema

def profile_schema():
    schema = pd.DataFrame({
        "sid": pd.Series(dtype="Int64"),
        "profile_id": pd.Series(dtype="Int64"),
        "profile_name": pd.Series(dtype="string"),
        "firstname_lastname": pd.Series(dtype="string"),
        "description": pd.Series(dtype="string"),
        "following": pd.Series(dtype="string"),
        "followers": pd.Series(dtype="string"),
        "n_posts": pd.Series(dtype="string"),
        "url": pd.Series(dtype="string"),
        "cts": pd.Series(dtype="string"),
        "is_business_account": pd.Series(dtype="string")
    })
    return schema

def post_schema():
    schema = pd.DataFrame({
        "sid": pd.Series(dtype="Int64"),
        "sid_profile": pd.Series(dtype="string"),
        "post_id": pd.Series(dtype="string"),
        "profile_id": pd.Series(dtype="string"),
        "location_id": pd.Series(dtype="string"),
        "cts": pd.Series(dtype="string"),
        "post_type": pd.Series(dtype="string"),
        "description": pd.Series(dtype="string"),
        "numbr_likes": pd.Series(dtype="string"),
        "number_comments": pd.Series(dtype="string")
    })
    return schema

In [57]:
#Reading csv and put the information inside of data frame 
#profile_file = r'E:\Kaggle\input\Instagram\instagram_profiles.csv'
#cities_file  = r'E:\Kaggle\input\Instagram\instagram_locations.csv'
posts_file   = r'E:\Kaggle\input\Instagram\instagram_posts.csv'
ec = 'latin'
delimiter = '\t'

print(f"Iniciando o processamento do arquivo {cities_file}....")
df_cities   = pd.read_csv(cities_file
,delimiter=delimiter
,encoding=ec)
print("Arquivo processado com sucesso....")

print(f"Iniciando o processamento do arquivo {profile_file}....")
df_profiles = pd.read_csv(profile_file
,delimiter=delimiter
,encoding=ec)
print("Arquivo processado com sucesso....")

print(f"Iniciando o processamento do arquivo {posts_file}....")
df_posts = pd.DataFrame()
df_cities_posts = pd.DataFrame()
df_profile_posts = pd.DataFrame()

count = 0
print("Arquivo processado com sucesso....")

for df_post_part in pd.read_csv( posts_file
    ,delimiter=delimiter #delimter
    ,encoding=ec #encoding
    ,error_bad_lines=False #Exlude badlines
    ,names=['sid','sid_profile','post_id','profile_id','location_id','cts','post_type','description','numbr_likes','number_comments'] 
    ,header=0#header
    ,quotechar='"'
    ,chunksize=1000000 #Leitura de arquivo particionado,quantidade de linhas
    #,dtype=post_schema() #Aplica scheam
    ):
    
    count = count + 10000
    
    print(f"Processando os dados.... Arquivo {posts_file}")
    print(f"Criando chunk... {count}")
    df_post_part = df_post_part.astype(post_schema().dtypes) #Aplica o schema em cada pedaco(chunk do arquivo)
    df_post_part = df_post_part.dropna(subset=['numbr_likes'])
    df_post_part = df_post_part.dropna(subset=['number_comments'])
    
    #print("Convertendo as colunas para sumarizacao.....")
    df_post_part['numbr_likes'] =  df_post_part['numbr_likes'].astype(float)
    df_post_part['number_comments'] =  df_post_part['number_comments'].astype(float)
    df_post_part['numbr_likes'] = df_post_part['numbr_likes'].apply(lambda x: int(round(x)))
    df_post_part['number_comments'] = df_post_part['number_comments'].apply(lambda x: int(round(x)))
    #Removendo linhas nulas para realizar o relacionamento
    #Observe que o parâmetro errors='coerce' é usado no método to_numeric(). 
    # Isso faz com que os valores que não puderem ser convertidos em números sejam convertidos em NaN
    #df_posts['sid'] = pd.to_numeric(df_posts['sid'],errors='coerce') 
    
    
    df_post_part['location_id'] = pd.to_numeric(df_post_part['location_id'],errors='coerce') 
    df_post_part['profile_id'] =  pd.to_numeric(df_post_part['profile_id'],errors='coerce') 
    df_post_part['location_id'] = df_post_part['location_id'].apply(Decimal)
    df_post_part['profile_id'] = df_post_part['profile_id'].apply(Decimal)
    
    df_post_part = df_post_part.dropna(subset=['location_id'])
    #print("Colunas convertidas com sucesso..")
    #print("Concatenando Data Frames.... ")
    df_cities_posts = pd.concat([df_cities_posts,df_post_part])# Reune todos os pedacos no data frame principal
        
    print("Agrupando posts por cidade...")
    df_cities_posts = df_post_part.groupby(['location_id','post_type']).agg({'post_id':'count','numbr_likes':'sum','number_comments':'sum'})
    df_cities_posts = df_cities_posts.sort_values(by='location_id', ascending =False).rename(columns={'location_id':'id','post_id':'qtd_posts','numbr_likes':'qtd_likes','number_comments':'qtd_comments'})
    df_profile_posts = pd.concat([df_profile_posts,df_post_part])# Reune todos os pedacos no data frame principal
    
    print("Agrupando posts por Perfis...")
    df_profile_posts = df_post_part.groupby(['profile_id','post_type']).agg({'post_id':'count','numbr_likes':'sum','number_comments':'sum'})
    df_profile_posts = df_profile_posts.sort_values(by='profile_id', ascending =False).rename(columns={'profile_id':'id','post_id':'qtd_posts','numbr_likes':'qtd_likes','number_comments':'qtd_comments'})

    
    
print("Dados de cidade agrupados")
print(df_cities_posts.head())
print(df_profile_posts.head())

# salvar o DataFrame como arquivo CSV com SOH como delimitador
df_cities_posts.to_csv('E:\Kaggle\output\TheMostPostsPerCityGroup.csv',sep='\x01')
df_profile_posts.to_csv('E:\Kaggle\output\TheMostPostsPerProfileGroup.csv',sep='\x01')


    #df_profiles_posts
    #df_cities.groupby('city').agg({'id':'count'})
    #df_posts = pd.concat([df_posts,df_post_part])# Reune todos os pedacos no data frame principal
   

Iniciando o processamento do arquivo E:\Kaggle\input\Instagram\instagram_locations.csv....
Arquivo processado com sucesso....
Iniciando o processamento do arquivo E:\Kaggle\input\Instagram\instagram_profiles.csv....
Arquivo processado com sucesso....
Iniciando o processamento do arquivo E:\Kaggle\input\Instagram\instagram_posts.csv....
Arquivo processado com sucesso....
Processando os dados.... Arquivo E:\Kaggle\input\Instagram\instagram_posts.csv
Criando chunk... 10000
Agrupando posts por cidade...
Agrupando posts por Perfis...
Processando os dados.... Arquivo E:\Kaggle\input\Instagram\instagram_posts.csv
Criando chunk... 20000
Agrupando posts por cidade...
Agrupando posts por Perfis...
Processando os dados.... Arquivo E:\Kaggle\input\Instagram\instagram_posts.csv
Criando chunk... 30000
Agrupando posts por cidade...
Agrupando posts por Perfis...
Processando os dados.... Arquivo E:\Kaggle\input\Instagram\instagram_posts.csv
Criando chunk... 40000
Agrupando posts por cidade...
Agrupando

## Aplicando schema nos dataframes


In [58]:
df_cities = df_cities.astype(location_schema().dtypes)
df_profiles = df_profiles.astype(profile_schema().dtypes)
df_posts = df_posts.astype(post_schema().dtypes)

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'

## The most profiles per city

In [59]:
city_group = df_cities.groupby('city').agg({'id':'count'})
city_group = city_group.sort_values(by='id', ascending =False).rename(columns={'id':'QTD_PROFILES','city':'CITY'})

# salvar o DataFrame como arquivo CSV com SOH como delimitador
#df.to_csv('exemplo.csv', sep='\x01', index=False)
city_group.to_csv('E:\Kaggle\output\TheMostProfilesPerCity.csv',sep='\x01')

city_group.head()

,QTD_PROFILES
city,
"New York, New York",29954
"Moscow, Russia",21087
"London, United Kingdom",20251
"Saint Petersburg, Russia",8636
"Los Angeles, California",7175


## The most posts per city

In [60]:
#Fazendo join pelo location_id
print("*"*25)
print("The most posts per city")
print("*"*25)
merged_df = pd.merge(df_cities_posts,df_cities, left_on='location_id', right_on='id')
print(merged_df[['city','qtd_posts','qtd_likes','qtd_comments']].sort_values(by='qtd_posts', ascending =False))

# salvar o DataFrame como arquivo CSV com SOH como delimitador
merged_df.to_csv('E:\Kaggle\output\TheMostPostsPerCity.csv',sep='\x01')

*************************
The most posts per city
*************************
                                             city  qtd_posts  qtd_likes  \
40866                                      Soroca         50       2232   
38850                 Comrat, GÄgÄuzia, Moldova         48       1992   
43523                          New York, New York         45       2872   
30891  Tiraspolul Nou, StÃ®nga Nistrului, Moldova         42       1149   
39768                 Centru, ChiÅinÄu, Moldova         41       1531   
...                                           ...        ...        ...   
33164                           Chisinau, Moldova          1         42   
33170                          Brooklyn, New York          1         43   
8121                      Victorville, California          1         60   
8117                     Nagoya-shi, Aichi, Japan          1        386   
24712                                Aosta, Italy          1        113   

       qtd_comments  
4

## The most posts per profile

In [62]:
print("*"*50)
print("The most posts per profile")
print("*"*50)
merged_df = pd.merge(df_profile_posts,df_profiles, left_on='profile_id', right_on='profile_id')
df_posts_profile = merged_df[['profile_name','qtd_posts','qtd_likes','qtd_comments']].sort_values(by='qtd_posts', ascending =False)
print(merged_df[['profile_name','qtd_posts','qtd_likes','qtd_comments']].sort_values(by='qtd_posts', ascending =False))

# salvar o DataFrame como arquivo CSV com SOH como delimitador
df_posts_profile.to_csv('E:\Kaggle\output\TheMostPostsPerProfile.csv',sep='\x01')

**************************************************
The most posts per profile
**************************************************
                        profile_name  qtd_posts  qtd_likes  qtd_comments
12446               wellnessacademie         38       1914            91
14276                 propertpix_ltd         34        631            17
22982                        balesii         32        230            11
9572   therapeutic.massage.northwich         32        465             3
17304                    writopialab         31       1175            43
...                              ...        ...        ...           ...
16879       thompsonbrotherslacrosse          1       1162             2
16880                   ilopushynska          1        139             1
16881                      ragnabley          1        170             5
16882                tvscalumstewart          1         12             0
22373                  sweet.lanaaaa          1        518          

In [ ]:
# abrir o arquivo
with open('E:\Kaggle\input\Instagram\instagram_posts.csv', 'r') as f:
    # contar o número de linhas
    num_linhas = sum(1 for linha in f)

# imprimir o número de linhas
print(f'O arquivo tem {num_linhas} linhas.')
